In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import tensorflow as tf
import numpy as np

import Audio_functions as af
from functools import partial
import datetime

import librosa
import soundfile as sf
import IPython.display as ipd

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [21]:
#data_folder = 'C:/Users/Toby/Speech_Data/CHiME3/data/audio/16kHz/isolated/et05_bth'
data_folder = 'C:/Users/Toby/Speech_Data/LibriSpeech/dev-clean'
sample_rate = 16384
n_parallel_readers = 4
n_channels = 1

In [22]:
data_pipe = tf.data.Dataset.list_files(data_folder + '/*.flac').map(partial(af.read_audio,
                                                                            sample_rate=sample_rate,
                                                                            n_channels=n_channels), 
                                                                    num_parallel_calls=n_parallel_readers)

In [23]:
sess = tf.Session()
next_file = data_pipe.make_one_shot_iterator().get_next()

In [24]:
next_file

<tf.Tensor 'IteratorGetNext_4:0' shape=<unknown> dtype=float32>

In [25]:
length = 0
file_count = 0
max_file_length = 0
min_file_length = 100000

while True:
    try:
        file = sess.run(next_file)
        len_file = len(file) / sample_rate
        length += len_file
        if len_file > max_file_length:
            max_file_length = len_file
        if len_file < min_file_length:
            min_file_length = len_file
        file_count += 1
        if file_count % 10 == 0:
            print('{fc} files processed'.format(fc=file_count))
    except tf.errors.OutOfRangeError:
        mean_file_length = length / file_count
        print('Files in folder: {fc}'.format(fc=file_count))
        print('Total length of audio in folder = {l}s ({lh} hours)'.format(l=round(length, 2), 
                                                                           lh=str(datetime.timedelta(seconds=length))))
        print('Maximum file length = {maxfl}s'.format(maxfl=round(max_file_length, 2)))
        print('Minimum file length = {minfl}s'.format(minfl=round(min_file_length, 2)))
        print('Mean file length = {meanfl}s'.format(meanfl=round(mean_file_length, 2)))
        break

InvalidArgumentError: TypeError: a bytes-like object is required, not 'str'
Traceback (most recent call last):

  File "C:\Program Files\Anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 157, in __call__
    ret = func(*args)

  File "C:\Users\Toby\MSc_Project\MScFinalProject\Audio_functions.py", line 10, in read_audio_py
    mono, _ = librosa.load(py_path, sr=sample_rate, mono=True)

  File "C:\Program Files\Anaconda3\lib\site-packages\librosa\core\audio.py", line 112, in load
    with audioread.audio_open(os.path.realpath(path)) as input_file:

  File "C:\Program Files\Anaconda3\lib\site-packages\audioread\__init__.py", line 111, in audio_open
    return ffdec.FFmpegAudioFile(path)

  File "C:\Program Files\Anaconda3\lib\site-packages\audioread\ffdec.py", line 127, in __init__
    stdin=self.devnull,

  File "C:\Program Files\Anaconda3\lib\site-packages\audioread\ffdec.py", line 89, in popen_multiple
    return subprocess.Popen(cmd, *args, **kwargs)

  File "C:\P

In [28]:
print('Files in folder: {fc}'.format(fc=file_count))
print('Total length of audio in folder = {l}s ({lh} hours)'.format(l=round(length, 2), 
                                                                   lh=str(datetime.timedelta(seconds=length))))
print('Maximum file length = {maxfl}s'.format(maxfl=round(max_file_length, 2)))
print('Minimum file length = {minfl}s'.format(minfl=round(min_file_length, 2)))
print('Mean file length = {meanfl}s'.format(meanfl=round(mean_file_length, 2)))

Files in folder: 7138
Total length of audio in folder = 54529.84s (15:08:49.844727 hours)
Maximum file length = 20.39s
Minimum file length = 1.3s
Mean file length = 7.64s


In [70]:
flac_path = 'C:/Users/Toby/Speech_Data/LibriSpeech/dev-clean/84/121123/84-121123-0001.flac'

In [74]:
wav_path = 'C:/Users/Toby/Speech_Data/CHiME3/data/audio/16kHz/isolated/et05_bth/F05_440C020E_BTH.CH1.wav'

In [78]:
tf_audio = af.read_audio(wav_path, sample_rate)
tf_audio = sess.run(tf_audio)
tf_audio

array([[0.00196944],
       [0.00017   ],
       [0.00072503],
       ...,
       [0.00056209],
       [0.00186147],
       [0.        ]], dtype=float32)

In [79]:
lib_audio = librosa.load(wav_path, sr=sample_rate, mono=True)
lib_audio

(array([0.00196944, 0.00017   , 0.00072503, ..., 0.00056209, 0.00186147,
        0.        ], dtype=float32), 16384)

In [80]:
sf_audio = sf.read(wav_path)
sf_audio

(array([ 2.13623047e-03, -3.05175781e-05,  9.76562500e-04, ...,
         7.62939453e-04,  1.40380859e-03,  3.23486328e-03]), 16000)

In [73]:
ipd.Audio(audio[0], rate=audio[1])